#### Importing Computer Vision Libraries

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

import time


In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

#### Preparing the data

In [ ]:
#training data

train_data = datasets.FashionMNIST(root='data', train= True, transform= ToTensor(), download= True)
test_data = datasets.FashionMNIST(root='data', train= False, transform= ToTensor(), download= True)

In [ ]:
print(f" the length of training data is {len(train_data)}, length of test data  { len(test_data)}")

In [ ]:
#visualize any image randomly
img, label = train_data[2]
img, label

In [ ]:
classnames = train_data.classes
classnames

In [ ]:
#visualize the image

plt.imshow(img.squeeze(), cmap="gray")
plt.title(classnames[label])
plt.axis(False)



In [ ]:
#multiple random images
torch.manual_seed(42)
fig  = plt.Figure(figsize=(9,9))
rows, cols = 4, 4

for i in range(1, rows*cols +1):
    #pick image randomly
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap = 'gray')
    plt.axis(False)
    plt.title(classnames[label])

In [ ]:
#dataloader to loop through the dataset

#hyperparameter
BATCH_SIZE = 32

train_data_batch = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle= True )

test_data_batch = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle= False)

print(f" The reduced training databatch is {len(train_data_batch)}")
print(f" The reduced test databatch is {len(test_data_batch)}")

In [ ]:
# Check out what's inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_data_batch))
train_features_batch.shape, train_labels_batch.shape

#### Creating a baseline model

In [ ]:
class FashionMNISTVO(nn.Module):
    def __init__(self, input_shape: int, hidden_layers: int,  output_shape: int):
        super().__init__()

        self.linear_stack = nn.Sequential( nn.Flatten(),
                                          nn.Linear(in_features= input_shape, out_features= hidden_layers),
                                          nn.Linear(in_features= hidden_layers, out_features= output_shape),
                                          )
        

    #define the forward method
    def forward(self, x):
        return self.linear_stack(x)

In [ ]:
#create a model for the class

model_0 = FashionMNISTVO(input_shape= 784,
                         hidden_layers= 10,
                         output_shape=len(classnames)).to('cpu')

model_0

#### Setup loss, optimizer and evaluation metrics

In [ ]:
#loss function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr = 0.1)

In [ ]:
#import helper functions

import requests
from pathlib import Path

r = requests.get(url='https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')

if Path("helper_functions.py").is_file():
    print(" Skipping the download the file already exists")
else:
    print('Downloading file...........')

    with open('helper_functions.py', 'wb') as f:

        f.write(r.content)

In [ ]:
#time function

def my_run_time(start_time, end_time):

    return f" the elapsed time is { end_time - start_time} seconds"
    

In [ ]:
from tqdm.auto import tqdm

#set the manual seed
torch.manual_seed(42)
#set the timer
start_time = time.time()

#set the epochs
epochs = 3

for epoch in tqdm(range(epochs)):

    print(f" this is epoch number {epoch}")

    train_loss = 0

    for batch, (X,y) in enumerate(train_data_batch):

        #train model
        model_0.train()

        #do the forward pass
        y_train_pred = model_0(X)

        #calculate the loss
        loss = loss_fn(y_train_pred, y)
        train_loss += loss

        #zero gradient

        optimizer.zero_grad()

        #loss backward
        loss.backward()

        #optimze step
        optimizer.step()

        if batch % 500 == 0:
            print(f"the batches checked are {batch * len(X)}")

        train_loss /= len(train_data_batch)

        #testing

        test_loss, acc = 0, 0

    model_0.eval()

    with torch.inference_mode():

            #do forward pass

        for batch, (X,y) in enumerate(test_data_batch):
                #do the forward pass
            y_test = model_0(X)

                #calculate the loss
            t_loss = loss_fn(y_test, y)
            test_loss += t_loss

        test_loss /= len(test_data_batch)

    print(f" the training loss is {train_loss:.6f}        <||||     test loss {test_loss:.6f}")

end_time = time.time()


print(my_run_time(start_time, end_time))

#### Adding Non-Linearity to our Model


In [ ]:
class FashionMNISTV1(nn.Module):

    def __init__(self, input_shape: int, hidden_layers : int, output_shape: int ):
        super().__init__()

        self.linear_stacktwo = nn.Sequential( nn.Flatten(),
                                             nn.Linear(in_features= input_shape, out_features= hidden_layers),
                                             nn.ReLU(),
                                             nn.Linear(in_features=hidden_layers, out_features= output_shape),
                                             nn.ReLU())
        

    def forward(self, x: torch.Tensor):
        return self.linear_stacktwo(x)
        

In [ ]:
#instantiate the class

model_1 = FashionMNISTV1(input_shape=784,
                         hidden_layers= 10,
                         output_shape=len(classnames)).to(device='cpu')

model_1

#### Loss function and Accuracy metrics

In [ ]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model_1.parameters(), lr = 0.1)



#### functionalizing the testing and training loop

In [ ]:
def training_step(model: torch.nn.Module,
                  optimizer: torch.optim.Optimizer,
                  loss: torch.nn.Module,
                  dataloader: torch.utils.data.DataLoader,
                  accuracy_fn,
                  device: torch.device = device):
    

    model.to(device)

    train_acc, train_loss = 0, 0

    for batch, (X,y) in enumerate(dataloader):

        model.train()

        #do the forward pass

        y_pred = model(X)

        #calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss = loss

        #accuracy function
        train_acc = accuracy_fn(y_true= y, y_pred= y_pred.argmax(dim=1))

        #optimizer

        optimizer.zero_grad()

        #loss backward
        loss.backward()

        #optimizer step
        optimizer.step()

        if batch % 440 == 0:

            print(f"the train loss is {train_loss:.5f}     || ")
            print(f"the train accuracy is {train_acc:.5f}     || ")

    
    

In [ ]:
def test_step(model: torch.nn.Module,
                  accuracy_fn,
                  loss_fn: torch.nn.Module,
                  dataloader: torch.utils.data.DataLoader,
                  device: torch.device = device):
    model.to(device)

    model.eval()
    test_loss = 0
    train_acc = 0


    #loop through the dataset data

    with torch.inference_mode():

        for batch, (X,y) in enumerate(dataloader):

            #forwward pass

            y_test = model(X)

            #calculate the loss

            loss = loss_fn(y_test, y)
            test_loss = loss

            train_acc = accuracy_fn(y_true= y, y_pred=y_test.argmax(dim=1)) 

            if batch % 440 == 0:
                print(f"the test loss is {test_loss:.5f}     || ")
                print(f" test accuracy is {train_acc}")

           


In [ ]:
#run a loop through all the data 

start_time = time.time()

epochs = 3

for epoch in tqdm(range(epochs)):

    #training data
    training_step(model=model_1,
                  optimizer= optimizer,
                  accuracy_fn= accuracy_fn,
                  loss = loss_fn,
                  dataloader=train_data_batch,
                  device= 'cpu' )
    
    test_step(model= model_1,
              loss_fn= loss_fn,
              accuracy_fn=accuracy_fn,
              dataloader= test_data_batch,
              device= 'cpu')


end_time = time.time()

print(my_run_time(start_time, end_time))

### Convolutional Neural Network

In [ ]:
class FashionMNISTV2(nn.Module):
    #class constructor
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels= input_shape,
                      out_channels= hidden_units,
                      kernel_size= 3,
                      stride= 1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d( in_channels = hidden_units,
                      out_channels = hidden_units,
                      kernel_size = 3,
                      stride = 1,
                      padding= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride = 2))
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels= hidden_units,
                      out_channels= hidden_units,
                      kernel_size= 3,
                      stride= 1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d( in_channels = hidden_units,
                      out_channels = hidden_units,
                      kernel_size = 3,
                      stride = 1,
                      padding= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride = 2)            

        )

        dummy_input = torch.zeros((1, input_shape, 28, 28))
        dummy_output = self.conv_block_2(self.conv_block_1(dummy_input))
        input_size = dummy_output.flatten(1).shape[1]

       
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= input_size, out_features= output_shape, )
        )

    def forward(self, x: torch.Tensor):

        x = self.conv_block_1(x)

        x = self.conv_block_2(x)

        x = x.flatten(1)

       

        x = self.classifier(x)

        return x

In [ ]:
## create an instance of a class

model_2 = FashionMNISTV2( input_shape=1,
                         hidden_units= 10,
                         output_shape= len(classnames))

model_2

In [ ]:
#pass random image through the model

rand_out = model_2(img.unsqueeze(1))
rand_out

##### Loss function and Optimizer

In [ ]:
from helper_functions import accuracy_fn


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_2.parameters(), lr=0.01 )


model_2.state_dict()

In [ ]:
## run the training and test loop

epochs = 3


start_time = time.time()

for epoch in tqdm(range(epochs)):

    print(f" this is the {epoch} number")

    training_step( model= model_2,
                  optimizer= optimizer,
                  loss = loss_fn,
                  accuracy_fn = accuracy_fn,
                  dataloader= train_data_batch,
                  device= 'cpu')
    

    test_step( model= model_2,
              loss_fn= loss_fn,
              accuracy_fn= accuracy_fn,
              dataloader= test_data_batch,
              device= 'cpu')
    
end_time = time.time()

print(my_run_time(start_time = start_time, end_time= end_time))

In [ ]:
## Evaluation model

torch.manual_seed(42)

def eval_model(model: torch.nn.Module,
               dataloader: torch.utils.data,
               accuracy_fn,
               loss_fn: nn.Module,
               device: torch.device = device):
    
    acc, loss = 0, 0

    model.eval()

    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            # define forward pass
            y_test = model(X)

            #loss fn and accuracy function
            loss += loss_fn(y_test, y)

            acc += accuracy_fn(y_true= y, y_pred= y_test.argmax(dim=1))

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [ ]:
#making predictions based on the 

def make_predictions(model: nn.Module,
                     data: list,
                     device: torch.device = device,
                     ):
    
    pred_probs = []

    model.eval()

    with torch.inference_mode():

        for sample in data:

            #make a forward pass

            y_preds = model(sample)

            #turn the predictions to probabilities

            pred_prob = torch.softmax(y_preds.squeeze(), dim=0)

            #append to list

            pred_probs.append(pred_prob)

    return torch.stack(pred_probs)

In [ ]:
import random

random.seed(42)

#prepare data
test_sample = []
test_label = []

for sample, label in random.sample(list(test_data), k = 9):
    test_sample.append(sample.unsqueeze(1))
    test_label.append(label)

In [ ]:
#make preduictions using our prediction function

predi_probs = make_predictions(model_2, data= test_sample, device= 'cpu')

#convert to prediction probs

prediction_probs = predi_probs.argmax(dim=1)

prediction_probs, test_label

In [ ]:
#making visuals of the data

# Plot predictions
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_sample):
  # Create a subplot
  plt.subplot(nrows, ncols, i+1)

  # Plot the target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # Find the prediction label (in text form, e.g. "Sandal")
  pred_label = classnames[prediction_probs[i]]

  # Get the truth label (in text form, e.g. "T-shirt")
  truth_label = classnames[test_label[i]] 

  # Create the title text of the plot
  title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
  if pred_label == truth_label:
      plt.title(title_text, fontsize=10, c="b") # green text if correct
  else:
      plt.title(title_text, fontsize=10, c="g") # red text if wrong
  plt.axis(False);